In [1]:
import os
import time
from dotenv import load_dotenv, dotenv_values
from pathlib import Path
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams, PointStruct
from qdrant_client.http import models
import openai
import tiktoken

load_dotenv()

_dotenv = dotenv_values(".env")
OPENAI_APIKEY = _dotenv['OPENAI_APIKEY']
DOC = "tec_navegacao.pdf"
EMBEDDING_MODEL = "text-embedding-ada-002"

openai.api_key = _dotenv['OPENAI_APIKEY'] #"sk-xxxxxxxxxxxxxxxxxxxxxxxxxx"

# Vector db setup

# cloud
#qdrant_client = QdrantClient(
#    host="<HOSTNAME>".
#    api_key="<API_KEY>",
#)


In [4]:
#netsh advfirewall firewall add rule name="Open Port 6333" dir=in action=allow protocol=TCP localport=6333
#qdrant_client = QdrantClient("localhost", port=6333)
#qdrant_client = QdrantClient(":memory:")
qdrant_client = QdrantClient(path="./qdrant")  # Persists changes to disk
#qdrant_client.recreate_collection(
#    collection_name="mycollection2",
#    vectors_config=models.VectorParams(size=1536, distance=models.Distance.COSINE),
#)
#print("Create collection reponse:", qdrant_client)

In [6]:
try:
    collection_info = qdrant_client.get_collection(collection_name="mycollection")
except Exception as e:
    print(e)
print("Collection info:", type(collection_info))

Collection info: <class 'qdrant_client.http.models.models.CollectionInfo'>


In [7]:
import pdfplumber

fulltext = ""
with pdfplumber.open(DOC) as pdf:
    for page in pdf.pages:
        fulltext += page.extract_text()

In [8]:
text = fulltext

chunks = []
while len(text) > 500:
    last_period_index = text[:500].rfind('.')
    if last_period_index == -1:
        last_period_index = 500
    chunks.append(text[:last_period_index])
    text = text[last_period_index+1:]
chunks.append(text)


print(chunks[0])

UNIVERSIDADE DO SUL DE SANTA CATARINA
JOSÉ STOFFEL FILHO
TECNOLOGIAS DE NAVEGAÇÃO AÉREA: UMA ABORDAGEM
SOBRE NAVEGAÇÃO DE ÁREA POR GPS E ADS-B
PALHOÇA
2020JOSÉ STOFFEL FILHO
TECNOLOGIAS DE NAVEGAÇÃO AÉREA: UMA ABORDAGEM
SOBRE NAVEGAÇÃO DE ÁREA POR GPS E ADS-B
Monografia apresentada ao Curso de
graduação em Ciências Aeronáuticas, da
Universidade do Sul de Santa Catarina,
como requisito parcial para obtenção do
título de Bacharel.
Orientador: Prof. MSc


In [6]:
len(chunks)

169

In [9]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    return len(encoding.encode(string))

def num_tokens_from_chunks(chunks: list[str]) -> int:
    return sum(num_tokens_from_string(chunk, "cl100k_base") for chunk in chunks)

num_tokens = num_tokens_from_chunks(chunks)
embedding_cost = num_tokens*.0001/1000
print(f"Number of tokens: {num_tokens}")
print(f"Cost in USD to generate tokens embeddings: {embedding_cost}")

Number of tokens: 19165
Cost in USD to generate tokens embeddings: 0.0019165


### Create embeddings

In [30]:
def generate_embeddings(chunks: list[str], _slice: int) -> list:
    #model RPM	TPM
    #text-embedding-ada-002	60	150 000
    points = []
    for i, chunk in enumerate(chunks):
        print(f"Embeddings {i}")
        response = openai.Embedding.create(
            input=chunk,
            model=EMBEDDING_MODEL
        )
        embeddings = response['data'][0]['embedding']

        points.append(PointStruct(id=i, vector=embeddings, payload={"text": chunk}))
        time.sleep(60)
        _id += _id
        print("---")
    return points

    #points = generate_embeddings(chunks, 5)


In [31]:
points = generate_embeddings(chunks)

operation_info = qdrant_client.upsert(
    collection_name="mycollection",
    wait=True,
    points=points
)
print(operation_info)

Embeddings 60 chunk:  No entanto, outros quesitos são de extrema importância para
que este sistema seja confiável, são eles: continuidade, disponibilidade, integridade, e
por fim, funcionalidade. Essas cinco características compõem os pilares da navegação
por GPS
---
Embeddings 120 chunk: 
Precisão: É o grau de conformidade entre a informação sobre a posição e
hora que proporciona o sistema de navegação e a posição e hora verdadeiras;
Integridade: É a garantia de que todas as funções do sistema de navegação
estão dentro dos limites de desempenho operacional
---
Embeddings 240 chunk:  É a capacidade do
sistema de navegação aérea de proporcionar aos usuários avisos oportunos
nos casos em que ele não deva ser utilizado;
Disponibilidade: É o percentual de tempo em que são utilizáveis as
informações providas por um sistema de navegação. É uma identificação da
capacidade desse sistema em proporcionar informações utilizáveis dentro de
uma determinada zona de cobertura, bem como do percentual 

### Final

In [19]:
def create_answer_with_context(query):
    response = openai.Embedding.create(
        input=query,
        model="text-embedding-ada-002"
    )
    embeddings = response['data'][0]['embedding']

    search_results = qdrant_client.search(
        collection_name="mycollection",
        query_vector=embeddings, 
        limit=5
    )

    prompt = "Contexto:\n"
    for result in search_results:
        prompt += result.payload['text'] + "\n---\n"
    prompt += f"Pergunta: {query}" + "\n---\nResposta:"

    print("----PROMPT START----")
    print(prompt)
    print("----PROMPT END----")

    generated = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "user", "content": prompt}
        ]
        )

    return generated.choices[0].message.content

### Answer with context

In [20]:
user_input = "como o RNAV nos ajuda?"
answer = create_answer_with_context(user_input)
print(answer)

----PROMPT START----
: Context:
ture from the aircraft manufacturer Airbus Industries. The use of
ADS-B system in the Brazilian territory, at Macaé terminal, the contingency procedures
during the loss of GPS signal and also the necessary equipment to enter the RNAV –
Area Navigation were presented
---
egação por
Satélite
GPS Global Positioning System – Sistema de Posicionamento Global
IAP Instrument Approach Procedure – Procedimento de Aproximação por
Instrumentos
ICAO International Civil Aviation Organization – Organização Internacional da
Aviação Civil
IFR Instrument Flight Rules – Regra de Voo por Instrumentos
INS Inertial Navigation System – Sistema de Navegação Inercial
JAA Joint Aviation Authority – Autoridade de Aviação Comum
LNAV Lateral Navigation – Navegação Lateral
MCDU Multipurpos
---
 Ao final da pesquisa, conclui-se que a navegação direta de um ponto a outro,
a economia de combustível, redução na emissão de poluentes e o aumento da
quantidade e qualidade na troca de infor

In [37]:
import json

with open('config.json') as f:
    # returns JSON object as 
    # a dictionary
    data = json.load(f)

    # Iterating through the json
    # list
    for i in data['openai']:
        print(i)
f.close()
data

completion-model
embedding-model


{'openai': {'completion-model': {'name': 'gpt-3.5-turbo',
   'rpm': 3500,
   'tpm': 90000,
   'pricing': 0.003},
  'embedding-model': {'name': 'text-embedding-ada-002',
   'rpm': 3000,
   'tpm': 1000000,
   'pricing': 0.0001}}}

In [57]:
from doc_reader import DocReader

reader = DocReader(DOC)
test = reader.parse_doc()
print(test)

None


In [53]:
reader.doc

'tec_navegacao.pdf'